In [ ]:
import os
import numpy as np
import rasterio
import matplotlib.image

from pathlib import Path


In [ ]:
files = {}

for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[0] not in files:
            files[parts[0]] = {}
        if parts[1] not in files[parts[0]]:
            files[parts[0]][parts[1]] = {}
        files[parts[0]][parts[1]][parts[2]] = f"data/{f}"


In [ ]:
dates = []
for f in files.values():
    dates += list(f.keys())
dates = sorted(list(set(dates)))
    

In [ ]:
dates = [d for d in dates if '2022' == d[:4] or '2023' == d[:4]]


In [ ]:
s = 0
c = 4550

sj = 200

offsets = {
    'T34UDG': (30000+s, 0),
    'T34UEG': (30000+s, 10000),
    'T34UFG': (30000+s, 20000),
    'T34VDH': (20000+s, 0),
    'T34VDJ': (10000+s, 0),
    'T34VDK': (0+s, 0), # top left
    'T34VEH': (20000+s, 10000),
    'T34VEJ': (10000+s, 10000),
    'T34VEK': (0+s, 10000),
    'T34VFH': (20000+s, 20000),
    'T34VFJ': (10000+s, 20000),
    'T34VFK': (0+s, 20000),
    'T35ULB': (30000+sj, 30980-c),
    'T35UMB': (30000+sj, 40980-c),
    'T35UNB': (30000+sj, 50980-c),
    'T35VLC': (20000+sj, 30980-c),
    'T35VLD': (10000+sj, 30980-c),
    'T35VLE': (0+sj, 30980-c),
    'T35VMC': (20000+sj, 40980-c),
    'T35VMD': (10000+sj, 40980-c),
    'T35VME': (0+sj, 40980-c),
    'T35VNC': (20000+sj, 50980-c),
    'T35VND': (10000+sj, 50980-c),
    'T35VNE': (0+sj, 50980-c)
}


In [ ]:
not_seen = {
    'T34UDG': 0,
    'T34UEG': 0,
    'T34UFG': 0,
    'T34VDH': 0,
    'T34VDJ': 0,
    'T34VDK': 0,
    'T34VEH': 0,
    'T34VEJ': 0,
    'T34VEK': 0,
    'T34VFH': 0,
    'T34VFJ': 0,
    'T34VFK': 0,
    'T35ULB': 0,
    'T35UMB': 0,
    #'T35UNB': 0,
    'T35VLC': 0,
    'T35VLD': 0,
    'T35VLE': 0,
    'T35VMC': 0,
    'T35VMD': 0,
    'T35VME': 0,
    #'T35VNC': 0,
    #'T35VND': 0,
    #'T35VNE': 0
}


In [ ]:
files = {k:v for k,v in files.items() if k in not_seen}


In [ ]:
rgb_composite = np.dstack(
    #(np.full((41680, 57410), 0.0), np.full((41680, 57410), 0.0), np.full((41680, 57410), 0.0))
    (np.full((41680, 47410), 0.0), np.full((41680, 47410), 0.0), np.full((41680, 47410), 0.0))
)


In [ ]:
for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            if ki in not_seen:
                del not_seen[ki]
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_b04 = rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)/10000
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    c_b08 = rasterio.open(fj["B08"], driver="JP2OpenJPEG").read(1)/10000           
                except BaseException as e:
                    if os.path.isfile(fj["B08"]):
                        print("REMOVED", fj["B08"])
                        os.remove(fj["B08"])
                    raise e

                mask = ((scl != 0) & (scl != 1) & (scl != 3) & (scl != 7) & (scl != 8) & (scl != 9) & (scl != 10))
                ndvi = ((c_b08-c_b04)/(c_b08+c_b04))
                clipped = np.clip(ndvi, 0.0, 1.0)
                clipped[clipped < 0.4] = 0.0
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = clipped[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
        else:
            print("no match", ki, d)
    #print("saving composite")
    if len(not_seen) > 0:
        print(f"missing cells, skipping save ({not_seen.keys()})")
    else:
        print("saving composite")
        matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/forest_frames/{d}.jpeg", rgb_composite)
